In [1]:
#下面是用来识别的函数
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import cv2
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist
import numpy as np
from std_msgs.msg import String
from sensor_msgs.msg import Joy
import time
from gazebo_msgs.srv import SpawnEntity
from ament_index_python.packages import get_package_share_directory
from sensor_msgs.msg import Image
from gazebo_msgs.srv import SetEntityState,GetEntityState
rclpy.init()
if_callback = 0
#取图工具
class LeftSubscriber(Node):
    def __init__(self):
        super().__init__('minimal_subscriber')
        self.subscription = self.create_subscription(
            Image,
            '/demo_cam/mycamera/left/image_demo',
            self.listener_callback,
            1)
        self.subscription  # prevent unused variable warning
        #self.recognizer = Recognizer()

    def listener_callback(self,msg):
        global if_callback
        self.left_img = np.array(msg.data)
        self.left_img = self.left_img.reshape((2800,4200,3))
        if_callback +=1
        print("In callback")

leftsubscriber = LeftSubscriber()#实例化
class Recognizer_big(object):
    def __init__(self,world):
        if world == 'world_1':
            self.tile_area = [3.4e7,4e7]
        elif world == 'world_2':
            self.tile_area = [1.7e7,2e7]
    
    def angle_cos(self,p0, p1, p2):
        d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
        return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )
    
    def find_squares(self,contours):
        squares = []#make a void list,to collect the contours which match the condition
        for cnt in contours:
            cnt_len = cv2.arcLength(cnt, True) #compute the perimeter of the contour
            cnt = cv2.approxPolyDP(cnt, 0.02*cnt_len, True) #Polygon approximation
            #if the approximated polygon has 4 sides,and area>1000 and a convex polygon
            if len(cnt) == 4 and cv2.contourArea(cnt) > self.tile_area[0] and cv2.contourArea(cnt)<self.tile_area[1] and cv2.isContourConvex(cnt):
                M = cv2.moments(cnt) #compute the moment of the contour
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])#compute the center
                cnt = cnt.reshape(-1, 2)
                max_cos = np.max([self.angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
                if max_cos < 0.1:# if nearly right angle
                    squares.append(cnt)
        return squares

    def compute_center(self,contour):
        M = cv2.moments(contour)  
        center_x = int(M["m10"] / M["m00"])
        center_y = int(M["m01"] / M["m00"])
        return center_x,center_y

    def imgcor2boxcor(self,x,y):#transformation from coordinate on the image to coordinate of the physic world 
        box_x = ((24090-y+6000)/10)/1000
        box_y = ((15060-x)/10)/1000
        return box_x,box_y
    def durch_translation(self,img):
        #perspective transform
        M = np.load('/home/jsy/expri_data/M_4200.npy')#load the matrix for perspective transfomation
        img = cv2.warpPerspective(img,M,(30121,24091))#project the image to a big picture, each pixel is 1mm 
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)#Grayscale
        ret, binary = cv2.threshold(gray,205,255,cv2.THRESH_BINARY)#Binarization，for this task when the thresh set between 200-210,with the best effect
        contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
        contours = self.find_squares(contours)
        #compute the angle of the conotours with the horizon
        angles = []
        centers = []
        w_h = []
        for i in contours:
            rect = cv2.minAreaRect(i)#fit a rectangle with the minimal area,in ideal,this rectangle should be the contour self
            angles.append(np.minimum(np.absolute(rect[2]),np.absolute(90+rect[2])))
            #sins.append(np.absolute(np.sin(np.deg2rad(rect[2]))))#之所以用sin而不是角度是因为正对着时会出现0和-90两种数字
        angle = np.mean(angles)#
        cos = np.cos(np.deg2rad(angle))
        sin = np.sin(np.deg2rad(angle))
        #compute how much it should translate,603 is the sollwert(603mm is the distance zwischen tiles)
        tx = 6030*cos#compute translation distance along X-axis
        ty = 6030*sin#compute translation distance along Y-axis
        #compute new contours after translation, if beyond the image, throw it
        new_contours = [] 
        for i in range(len(contours)):
            a_contour = contours[i]+np.array([tx,ty])
            if a_contour[a_contour[:,0]>30120].size>0 or a_contour[a_contour[:,1]>24090].size>0:
                pass
            else:
                new_contours.append(a_contour)

        new_contours = np.int32(new_contours)

        #compare all new contour with feature of ground, if the color of a contours'center is gray, that means there is no tile
        next_contours = []
        img_coordinate = []
        for i in new_contours:
            x,y = self.compute_center(i)
            #if color on center is gray, then save it in a list
            if np.std(img[y][x])<0.1:#attention! Numpy index of a image is image[y][x],x is horizon,y is vertical
                next_contours.append(i)
                img_coordinate.append((x,y))
        next_contours = np.int32(next_contours)
        #frome image coordinate to coordinate relative to the center of the Fliesenleger
        box_coordinate = []
        for i in img_coordinate:
            box_x,box_y = self.imgcor2boxcor(i[0],i[1])
            box_coordinate.append([box_x,box_y,-0.845,0.,0.,angle])

        for i in img_coordinate: 
            img = cv2.circle(img,i, 5, (255, 0, 0), -1)#draw the centers of the choosen contour
        cv2.drawContours(img,next_contours,-1,(0,0,255),3)#draw the choosen contour 
        self.img_contours = im.fromarray(img)
        
        return box_coordinate
    
class Set_State_Client(Node):
    def __init__(self):
        super().__init__('set_state_client')
        self.cli = self.create_client(SetEntityState,'/set_entity_state')
        #while not self.cli.wait_for_service(timeout_sec=1.0):
        #    self.get_logger().info('service not available, waiting again...')
        self.req = SetEntityState.Request()

    def send_request(self):
        self.req.state.name = 'nested_model'
        self.future = self.cli.call_async(self.req)
        
class Get_State_Client(Node):
    def __init__(self):
        super().__init__('get_state_client')
        self.cli = self.create_client(GetEntityState,'/get_entity_state')
        #while not self.cli.wait_for_service(timeout_sec=1.0):
        #    self.get_logger().info('service not available, waiting again...')
        self.req = GetEntityState.Request()
        self.req.name = 'nested_model'
        self.req.reference_frame = 'world'

    def send_request(self):
        self.future = self.cli.call_async(self.req)
    

In [2]:
#不过有些位置识别不了，所以会有很多为空
#而且这里有个问题，就是照片刷新率跟不上执行速度，也就是说调整姿势后，用subscriber得到的照片有时候会是上一时刻的
start = time.time()
result_list = []
points = np.load('/home/jsy/expri_data/可用的点.npy')
real_x = 1.206
real_y = 0.603
counter_1 = 0
counter_2 = 0
set_state_client = Set_State_Client()
get_state_client = Get_State_Client()
recognizer = Recognizer_big('world_1')
for pos in points:
    x = pos[0]
    y = pos[1]
    z = pos[2]
    #rclpy.spin_once(leftsubscriber)
    set_state_client.req.state.pose.position.x = x#给request的x赋值
    set_state_client.req.state.pose.position.y = y#给request的y赋值
    set_state_client.req.state.pose.orientation.z = np.sin(np.deg2rad(-180.+z)/2)#给出绕z轴角度转换成四元数的z为sin(alpha/2)
    set_state_client.req.state.pose.orientation.w = np.cos(np.deg2rad(-180.+z)/2)#给出绕z轴角度转换成四元数的w为cos(alpha/2)
    set_state_client.send_request()#发送调整位姿请求
    rclpy.spin_until_future_complete(set_state_client,set_state_client.future)#必须用until future complete才不会出错
    print(set_state_client.future.result().success)
    get_state_client.send_request()
    rclpy.spin_until_future_complete(get_state_client,get_state_client.future)#获取位资
    print(get_state_client.future.result().success)
    while if_callback<3:
        rclpy.spin_once(leftsubscriber)#每运行一次就取回一个图在leftsubscriber.left_img里
    if_callback=0
    box_coordinate = recognizer.durch_translation(leftsubscriber.left_img)
    counter_1+=1
    if box_coordinate!= []:
        angle_rad = np.deg2rad(-180.+z)
        expri_x = np.cos(angle_rad)*box_coordinate[0][0]-np.sin(angle_rad)*box_coordinate[0][1]+x
        expri_y = np.sin(angle_rad)*box_coordinate[0][0]+np.cos(angle_rad)*box_coordinate[0][1]+y
        error_x = expri_x-real_x#计算x方向的误差
        error_y = expri_y-real_y#计算y方向的误差
        error_center = np.sqrt(np.square(error_x)+np.square(error_y))#计算中心点距离的绝对误差
        error_angle = box_coordinate[0][5]-z
        cam_x = np.cos(angle_rad)*0.45-np.sin(angle_rad)*0.0315+x
        cam_y = np.sin(angle_rad)*0.45+np.cos(angle_rad)*0.0315+y
        result =[x,expri_x,y,expri_y,z,box_coordinate[0][5],error_x,error_y,error_center,error_angle,cam_x,cam_y]#返回每次实验的机器位置，旋转角度，以及测定的误差和摄像头的位置
        result_list.append(result)
        counter_2+=1
        print(result)
end = time.time()
print('测试完毕，共测试了 %d 次，其中识别出砖的共 %d 次' % (counter_1,counter_2))
print('共耗时 '+str(end-start)+'秒')

True
True
In callback
In callback
In callback
[2.0, 1.2056653040720402, 1.2500000000000004, 0.6028280515620017, 30.0, 29.983909606933594, -0.00033469592795976055, -0.00017194843799828163, 0.00037628131699952115, -0.01609039306640625, 1.6260385682970027, 0.9977201997807906]
True
True
In callback
In callback
In callback
[2.0, 1.205773009087924, 1.2500000000000004, 0.6030888106496001, 31.5, 31.488900320870535, -0.0002269909120760083, 8.881064960009066e-05, 0.00024374619104201037, -0.0110996791294653, 1.632770630829211, 0.9880174807006695]
True
True
In callback
In callback
In callback
[2.0, 1.205472719228855, 1.2500000000000004, 0.602867988652694, 33.0, 32.98422459193638, -0.0005272807711449445, -0.00013201134730600472, 0.0005435549718627859, -0.0157754080636181, 1.6397543740275327, 0.9784943113529573]
True
True
In callback
In callback
In callback
[2.0, 1.2054757911010041, 1.2500000000000004, 0.6027405454739, 34.5, 34.475843157087056, -0.0005242088989958482, -0.0002594545260999892, 0.00058

In callback
[2.0, 1.2057520344406623, 1.3100000000000005, 0.6030271793025899, 31.5, 31.493765694754465, -0.0002479655593377039, 2.717930258988588e-05, 0.0002494506626708635, -0.006234305245534699, 1.632770630829211, 1.0480174807006697]
True
True
In callback
In callback
In callback
[2.0, 1.2056657331353864, 1.3100000000000005, 0.6028982233902563, 33.0, 32.993355887276785, -0.0003342668646135749, -0.00010177660974364233, 0.0003494178230564372, -0.006644112723215301, 1.6397543740275327, 1.0384943113529572]
True
True
In callback
In callback
In callback
[2.0, 1.2054617407871122, 1.3100000000000005, 0.602909952943083, 34.5, 34.486663818359375, -0.0005382592128877484, -9.004705691695314e-05, 0.0005457393633575862, -0.013336181640625, 1.6469850115832252, 1.0291572184422322]
True
True
In callback
In callback
In callback
[2.0, 1.2054317414700573, 1.3100000000000005, 0.6028016243396106, 36.0, 35.97629383632115, -0.000568258529942689, -0.0001983756603893827, 0.0006018892418772325, -0.0237061636788

In callback
In callback
[2.0, 1.2056207274407718, 1.3700000000000006, 0.6029931543145977, 25.5, 25.47993755340576, -0.00037927255922820535, -6.845685402323554e-06, 0.0003793343348447911, -0.02006244659423828, 1.607397721592024, 1.147838569979247]
True
True
In callback
In callback
In callback
[2.0, 1.2055124415181457, 1.3700000000000006, 0.6027892275146671, 27.0, 26.97963523864746, -0.00048755848185422224, -0.0002107724853328552, 0.0005311669359075193, -0.020364761352539062, 1.6133477648570302, 1.137637569605271]
True
True
In callback
In callback
In callback
[2.0, 1.2054367812704547, 1.3700000000000006, 0.6029088832204192, 28.5, 28.482855115618026, -0.000563218729545234, -9.11167795807355e-05, 0.0005705415014104686, -0.017144884381973924, 1.6195628002502933, 1.1275958188343247]
True
True
In callback
In callback
In callback
[2.0, 1.205503779844974, 1.3700000000000006, 0.603107819729934, 30.0, 29.98823438371931, -0.0004962201550260392, 0.00010781972993401112, 0.0005077987164390133, -0.011

In callback
[2.02, 1.2055848191660041, 1.2500000000000004, 0.6028301589017535, 33.0, 32.99055916922433, -0.00041518083399583183, -0.00016984109824647042, 0.0004485767755591474, -0.009440830775666598, 1.6597543740275327, 0.9784943113529573]
True
True
In callback
In callback
In callback
[2.02, 1.2055288625018297, 1.2500000000000004, 0.6029855749811786, 34.5, 34.484073638916016, -0.00047113749817029316, -1.4425018821428992e-05, 0.0004713582749354906, -0.015926361083984375, 1.6669850115832252, 0.9691572184422321]
True
True
In callback
In callback
In callback
[2.02, 1.2053596136499451, 1.2500000000000004, 0.6027532688939767, 36.0, 35.980429331461586, -0.0006403863500548113, -0.00024673110602324577, 0.000686273207998081, -0.019570668538413827, 1.6744575879784867, 0.9600126011455766]
True
True
In callback
In callback
In callback
[2.02, 1.20540718043671, 1.2500000000000004, 0.6028826780900324, 37.5, 37.4776980082194, -0.0005928195632900479, -0.00011732190996760483, 0.0006043173546886188, -0.02

[2.02, 1.2056882866901195, 1.3100000000000005, 0.6029558687278945, 22.5, 22.491397857666016, -0.0003117133098804459, -4.413127210545209e-05, 0.00031482178567289194, -0.008602142333984375, 1.6163087384894212, 1.1086902501616045]
True
True
In callback
In callback
In callback
[2.02, 1.2057185556819192, 1.3100000000000005, 0.6028670426014215, 24.0, 23.99631690979004, -0.000281444318080748, -0.00013295739857843714, 0.00031126929501106053, -0.0036830902099609375, 1.6217167483177173, 1.0981918287001484]
True
True
In callback
In callback
In callback
[2.02, 1.2058011565269364, 1.3100000000000005, 0.6028960307796846, 25.5, 25.499797821044922, -0.0001988434730635369, -0.00010396922031541411, 0.00022438432555097214, -0.000202178955078125, 1.6273977215920241, 1.087838569979247]
True
True
In callback
In callback
In callback
[2.02, 1.2055117991642654, 1.3100000000000005, 0.6029682463288035, 27.0, 26.982545580182755, -0.0004882008357345313, -3.1753671196455e-05, 0.0004892324106662879, -0.0174544198172

...

[2.3800000000000003, 1.2057616994246871, 1.2200000000000004, 0.6030817124918407, 10.5, 10.500114440917969, -0.000238300575312822, 8.171249184074636e-05, 0.00025192081199703606, 0.00011444091796875, 1.9432757106492233, 1.1070214839402694]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2057059991338246, 1.2200000000000004, 0.6030691290511472, 12.0, 11.996902465820312, -0.00029400086617537013, 6.91290511471987e-05, 0.0003020187660136037, -0.0030975341796875, 1.9463827979305472, 1.095628089708894]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2055654373764566, 1.2200000000000004, 0.6030883790077827, 13.5, 13.50102318657769, -0.0004345626235433553, 8.83790077826907e-05, 0.00044345859197622594, 0.0010231865776901827, 1.9497870647825069, 1.0843199337723162]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2054739567866244, 1.2200000000000004, 0.6029860505519098, 15.0, 14.991344875759548, -0.000526043213375571, -1.394944809018916

In callback
[2.3800000000000003, 1.2054454876495426, 1.2500000000000004, 0.6029554130377272, 15.0, 14.992724100748697, -0.0005545123504573368, -4.45869622728301e-05, 0.0005563020258946026, -0.0072758992513026755, 1.9534861780906492, 1.1031047661757603]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2052667873057195, 1.2500000000000004, 0.6027018314617496, 16.5, 16.473641395568848, -0.0007332126942805051, -0.00029816853825037626, 0.0007915208981804861, -0.026358604431152344, 1.957477602667487, 1.0919902732348872]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2053593517678904, 1.2500000000000004, 0.6029470519958604, 18.0, 17.973814010620117, -0.0006406482321095197, -5.294800413957468e-05, 0.0006428325197494426, -0.026185989379882812, 1.961758602989992, 1.0809840722679767]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2055609384489014, 1.2500000000000004, 0.602844299489703, 19.5, 19.5050048828125, -0.0004390615510985185, 

In callback
[2.3800000000000003, 1.2053788825312202, 1.2800000000000005, 0.6027722610551212, 21.0, 20.98723384312221, -0.0006211174687797971, -0.00022773894487881918, 0.0006615526714011063, -0.012766156877791701, 1.9711773984869365, 1.0893266392699534]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2051421330248167, 1.2800000000000005, 0.602748671166663, 22.5, 22.48240852355957, -0.0008578669751833079, -0.0002513288333370056, 0.0008939250133969284, -0.017591476440429688, 1.9763087384894216, 1.0786902501616045]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2054844979240977, 1.2800000000000005, 0.6028377554947487, 24.0, 23.999658857073104, -0.00051550207590223, -0.00016224450525126333, 0.0005404310036847773, -0.00034114292689579884, 1.9817167483177176, 1.0681918287001484]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.205308289638207, 1.2800000000000005, 0.6030359200021876, 25.5, 25.482934134347097, -0.000691710361792941,

[2.3800000000000003, 1.2053431692867165, 1.3100000000000005, 0.6029156839113008, 28.5, 28.478280203683035, -0.000656830713283485, -8.431608869918783e-05, 0.0006622203475626681, -0.0217197963169653, 1.9995628002502936, 1.0675958188343246]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2054889790326702, 1.3100000000000005, 0.6029127623650112, 30.0, 29.992012568882533, -0.0005110209673298005, -8.723763498874604e-05, 0.0005184137671870942, -0.007987431117467025, 2.006038568297003, 1.0577201997807908]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2052690719952086, 1.3100000000000005, 0.6026732178200802, 31.5, 31.481454213460285, -0.0007309280047913536, -0.0003267821799197712, 0.0008006511982763697, -0.018545786539714726, 2.0127706308292113, 1.0480174807006697]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.20547340876948, 1.3100000000000005, 0.6028176426886668, 33.0, 32.98233286539713, -0.0005265912305199016, -0.000182357311

In callback
[2.3800000000000003, 1.2055907316172267, 1.3400000000000005, 0.6026848975257338, 36.0, 36.006985346476235, -0.0004092683827732202, -0.0003151024742661379, 0.000516517355397134, 0.006985346476234611, 2.034457587978487, 1.0500126011455766]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2053563050029565, 1.3400000000000005, 0.6027776998735784, 37.5, 37.48591995239258, -0.0006436949970434203, -0.0002223001264215485, 0.00068099970295571, -0.014080047607421875, 2.042166981882719, 1.0410667267269023]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2055101058697055, 1.3400000000000005, 0.6027762764358379, 39.0, 38.98120435078939, -0.0004898941302944326, -0.00022372356416211492, 0.000538561502577318, -0.01879564921060961, 2.050107909662433, 1.032325726241679]
True
True
In callback
In callback
In callback
[2.3800000000000003, 1.2053396750195615, 1.3400000000000005, 0.6027874045318746, 40.5, 40.48748524983724, -0.0006603249804384426, -0.00021

In [3]:
data_4k = np.vstack(result_list)

In [5]:
data_4k.shape

(7665, 12)

In [6]:
np.save('/home/jsy/expri_data/data_4k.npy',data_4k)